In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np

/home/admin2/mambaforge/envs/cct_flowers17_v5/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Implementacja nn.CrossEntropyLoss()

Logits <==> (-inf, inf) <br>
Predictions (preds) <==> (0, 1)

In [2]:
# batch_size=32, n_classes=5
logits = torch.randn((32, 5))
targets_dense = torch.randint(low=0, high=5, size=(1, 32)).squeeze()

In [3]:
targets_dense

tensor([0, 4, 0, 3, 2, 4, 2, 2, 1, 4, 1, 0, 2, 4, 2, 3, 4, 1, 0, 3, 1, 4, 4, 0,
        0, 2, 4, 4, 2, 0, 0, 2])

In [4]:
preds = torch.nn.Softmax(dim=1)(logits)

In [5]:
EXAMPLE_IDX = 13
print(preds[EXAMPLE_IDX, :].shape)
print(preds[EXAMPLE_IDX, :].sum())

torch.Size([5])
tensor(1.0000)


In [6]:
targets_sparse = F.one_hot(targets_dense)

In [7]:
AVG_HEAD_SIZE = 5
targets_sparse[:AVG_HEAD_SIZE, :]

tensor([[1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0]])

In [8]:
explicit_ce_loss = [
    sum([
        -(y_i * torch.log(p_i))
        for p_i, y_i in zip(distribution, sparse_target)
    ])
    for distribution, sparse_target in zip(preds, targets_sparse)
]
explicit_ce_loss = torch.Tensor(explicit_ce_loss)

In [9]:
ce_loss = nn.CrossEntropyLoss(reduction="none")(logits, targets_dense)
torch.isclose(explicit_ce_loss, ce_loss).all().item()

True

# Implementacja nn.NLLLoss() oraz nn.LogSoftmax()

In [10]:
ce_loss_v2 = nn.NLLLoss(reduction="none")(torch.nn.LogSoftmax(dim=1)(logits), targets_dense)
torch.isclose(ce_loss, ce_loss_v2).all().item()

True

Jak widać NLLLoss to na 99% iloczyn skalarny pomnożony przez -1.